In [ ]:
# !! pip install fastai

In [ ]:
# ones = u.read_ones(data_path / 'train.csv')
# u.add_more_time(ones)
# ones = ones.rename(columns={'sid': 'segment_id', 'day': 'weekday'})

# vehinj = u.read_vehinj(Path('../data/SANRAL_v2'))

# field_combs = [
#     tuple([('segment_id')]), tuple(['hour']), tuple(['weekday']), tuple(['month']),
#     ('segment_id', 'hour'), ('segment_id', 'weekday'), ('segment_id', 'month'), ('weekday', 'month'), ('hour', 'weekday'),
#     ('segment_id', 'weekday', 'hour'), ('segment_id', 'weekday', 'month')
# ]


# train = u.select_by_time(train_full, '2017-01-01', '2018-10-01')
# u.add_statistic(data=train, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2017-01-01',
#                )

# val = u.select_by_time(train_full, '2018-10-01', '2019-01-01')
# u.add_statistic(data=val, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2018-10-01',
#                )

# u.add_statistic(data=test, stat_data=ones,
#                 field_combs=field_combs, prefix='hist_',
#                 tstart='2016-01-01', tend='2019-01-01',
#                )

# x_cols = cols
# x_cols.extend([c for c in train.columns if c.startswith('hist_')])

# train = u.select_by_time(train_full, '2016-01-01', val_tstart, time_col='datetime')
# val = u.select_by_time(train_full, val_tstart, '2019-01-01', time_col='datetime')

# def prepare(df):
#     df[cat_cols] = df[cat_cols].replace(np.nan, 'NAN').astype('object')
#     df[cat_cols] = df[cat_cols].astype('str')
    

# prepare(train_full)
# prepare(test)

# val = train_full.loc[ids_val]
# val.reset_index(inplace=True)

# u.run_many_th_experiment(val, np.array(predictions), th_score=0.12)

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import catboost
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool, cv
from fastai.tabular import *
from fastai.callbacks.tracker import EarlyStoppingCallback, SaveModelCallback
from pathlib import Path
import src.data_utils as u
import src.fai_utils as f
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

u.random_seed(42)

data_path = Path('../data')

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')


In [ ]:
train_full = pd.read_pickle(data_path / 'train_v2q_opt.pkl')
test = pd.read_pickle(data_path / 'test_v2q_opt.pkl')

x_cols = train_full.columns.values.tolist()

cat_cols = [
       'segment_id', 'weekday', 'month', 'hour', 'dayofyear',
       'weekofyear', 'ROADNO', 'CLASS', 'SURFTYPE', 
       'PAVETYPE', 'CONDITION', 'vds_id','vms_count', 'vds_count','wind_dir', 'cloud_cover',
       'weather_cond', 'cloud_1', 'cloud1_cover', 'cloud_height', 'cloud_2', 'precip_mm',
       'cloud_3', 'precip_time', 'public_holiday', 'school_holiday'
]

cont_names = list(set(x_cols) - set(cat_cols))
cont_names.remove('datetime x segment_id')
cont_names.remove('datetime')
cont_names.remove('y')

In [ ]:
for f in cont_names:
    test[f] = test[f].fillna(0)
    train_full[f] = train_full[f].fillna(0)

In [ ]:
val_tstart = '2018-10-01'
ids_val = train_full[train_full.datetime >= pd.Timestamp(val_tstart)].index

# Fast ai

In [ ]:
procs = [FillMissing, Categorify, Normalize]

test_tab = TabularList.from_df(test, cat_names=cat_cols, cont_names=cont_names)

data = (TabularList.from_df(train_full, procs=procs, cat_names=cat_cols, cont_names=cont_names)
                   .split_none()
                   .label_from_df(cols='y')
                   .add_test(test_tab)
                   .databunch(bs=200_000))

In [ ]:
weights = torch.tensor([1, 10]).float().to(device)

learn = tabular_learner(data, path='../results/fai', layers=[128],
                        metrics=u.F1(th_start=0, th_stop=1, steps=20),
                        callback_fns=[ShowGraph,
                                      partial(EarlyStoppingCallback,
                                              monitor='f1', min_delta=0.001, patience=5)
                                      ],
                        loss_func=torch.nn.CrossEntropyLoss(weight=weights),
                        opt_func=torch.optim.Adam
                       )

### Fit with one fold validation

In [ ]:
fit_func = lambda learn: learn.fit(1, slice(5e-03),
                                   callbacks=[
                                       SaveModelCallback(learn, every='improvement',
                                                         monitor='f1', name='best')
                                   ])

In [ ]:
learn.data.split_by_idx(ids_val)

learn.lr_find()
learn.recorder.plot()
plt.show()

fit_func(learn)

learn.recorder.plot_losses()
learn.recorder.plot_lr()

In [ ]:
predictions, *_ = learn.get_preds(DatasetType.Valid)

### Fit with K-fold validation

In [ ]:
predictions_arr, scores = f.fit_predict_cv(val_ids=ids_val, learn=learn, fit_func=fit_func)

In [ ]:
probas = predictions_arr.mean(axis=0)

In [ ]:
u.estimate(learn, validation_pool=None, y_true=train_full.y[ids_val],
           th_start=0, th_stop=1, steps=20)

# Catboost

In [ ]:
x_cols = cont_names + cat_cols

train = train_full[train_full.datetime < pd.Timestamp(val_tstart)]
val = train_full[train_full.datetime >= pd.Timestamp(val_tstart)]

In [ ]:
def prepare(df):
    df[cat_cols] = df[cat_cols].replace(np.nan, 'NAN').astype('object')
    df[cat_cols] = df[cat_cols].astype('str')

    
prepare(train)
prepare(val)
prepare(test)

In [ ]:
train_pool = Pool(data=train[x_cols], label=train['y'], cat_features=cat_cols)
val_pool =  Pool(data=val[x_cols], label=val['y'], cat_features=cat_cols)
test_pool = Pool(data=test[x_cols], cat_features=cat_cols)

In [ ]:
# --bootstrap-type
# --bagging-temperature
# --random-strength
# --depth
# --grow-policy
# --max-leaves
# --one-hot-max-size
# --rsm
# --boosting-type
# --boost-from-average
# --model-shrink-rate

params = {
    'iterations': 350,
    'learning_rate': 0.05,
    'scale_pos_weight': 10,
    'has_time': False,
    'depth': 10,
    'one_hot_max_size': 1000,
    
    'loss_function': 'Logloss',
    'task_type': 'CPU',
    'use_best_model': True,
    'eval_metric': m.FlexibleF1(0, 1, 20)
}

In [ ]:
cv(pool=train_pool,
   params=params,
   plot=True,
   early_stopping_rounds=10,
   fold_count=3,
   stratified=True,
   logging_level='Verbose'
  )

In [ ]:
cls = CatBoostClassifier(**params)

cls.fit(
    train_pool,
    eval_set=val_pool,
    plot=True,
    verbose=1,
    early_stopping_rounds=30
)

In [ ]:
# cls = CatBoostClassifier()
# cls.load_model(str(model_path))

cls.save('../results/model2.pt')

In [ ]:
th = u.estimate(cls, val_pool)

# Attemption of postprocessing

In [ ]:
gt = val['y'].astype(bool)
probas = cls.predict_proba(val_pool)[:, 1] 
pred = probas > th

n_th = 0
w_events = (val.cnt_event_with_veh > n_th) | (val.cnt_event_with_inj > n_th)

In [ ]:
vehinj = vehinj.drop_duplicates('EventID', keep='last')
vehinj.time = vehinj.time.dt.floor('H')

In [ ]:
def probas_to_pred(data_fixed_t: pd.DataFrame, n_vehinj: int) -> np.ndarray:
    probas = np.array(data_fixed_t['probas'])
    
    pred = probas > th
    
    if abs(n_vehinj - pred.sum()) > 15:
        pred = np.zeros(len(data_fixed_t))
        
        ii_top = np.argsort(probas)[-round(3 * n_vehinj):]
        pred[ii_top] = 1
        
    return pred.astype(bool)


def postproc(data: pd.DataFrame, vehinj: pd.DataFrame) -> np.ndarray:
    assert {'time', 'probas'} - set(data.columns) == set()

    # get time -> n_events mapping from raw table
    vehinj = vehinj.drop_duplicates('EventID', keep='last')
    vehinj.time = vehinj.time.dt.floor('H')
    time_to_nevents = Counter(vehinj.time)

    pred = np.zeros(len(data))
    for t in tqdm(set(data.time)):

        wt = data.time == t
        data_t = data[wt]

        n = time_to_nevents[t]
        if n == 0:
            continue
        else:
            pred[wt] = probas_to_pred(n_vehinj=n, data_fixed_t=data_t)
            
    return pred



val['probas'] = probas
pred3 = postproc(data=val, vehinj=vehinj)

print(sum(pred3))
u.scores(val_pool.get_label(), pred3);

In [ ]:
time_to_npred   = Counter(val.time[pred])
time_to_ngt     = Counter(val.time[gt])
time_to_nvehinj = Counter(vehinj.time)

data_events             = pd.DataFrame(data={'time': val.time.unique()})
data_events['n_pred']   = data_events.time.apply(lambda t: time_to_npred[t])
data_events['n_gt']     = data_events.time.apply(lambda t: time_to_ngt[t])
data_events['n_vehinj'] = data_events.time.apply(lambda t: time_to_nvehinj[t])

In [ ]:
data_events[1500:1550]

In [ ]:
Counter(val[val.y.astype(bool)].segment_id).most_common(n=20)

In [ ]:
print('veh/inj event')
u.plot_events(val.time[w_events])

In [ ]:
w_sid = val.segment_id == 'AJRKP0C'

print('predict')
u.plot_events(val.time[w_sid & pred])
print('groind truth')
u.plot_events(val.time[w_sid & gt])

In [ ]:
from collections import defaultdict

sid_to_f1 = defaultdict(lambda : None)
for sid in tqdm(set(val.segment_id)):
    w_sid = val.segment_id == sid
    f = f1_score(y_true=val_pool.get_label().astype(bool)[w_sid], y_pred=pred[w_sid])
    sid_to_f1[sid] = f
    

In [ ]:
cnt_pred = Counter(val.segment_id[pred])

pred_info = [(sid, cnt_pred[sid], sid_to_f1[sid]) for sid in set(val.segment_id)]

from pprint import pprint
pprint(sorted(pred_info, key=lambda tri: tri[2], reverse=True)[:30])

sids, cnts, f1s = list(zip(*pred_info))
plt.scatter(cnts, f1s)
plt.xlabel('n pred')
plt.ylabel('f1')
plt.show()

In [ ]:
# Compare top-pred and top-gt 

n_top = 25
top_true = set([k for (k, _) in Counter(val.segment_id[val.y.astype(bool)]).most_common(n_top)])
top_pred = set([k for (k, _) in Counter(val.segment_id[pred]).most_common(n_top)])

iou_top = len(top_true.intersection(top_pred)) / len(top_true.union(top_pred))
print(f'iou for top sids: {iou_top}')